In [13]:
import pandas_datareader.data as pdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [14]:
start = datetime.datetime(2020,1,1)
end = datetime.datetime.today()
df = pd.DataFrame()                                          #Creates Dataframe
df['2Y'] = pdr.DataReader('DGS2', 'fred', start, end)        #Fetches 2s Data from fred
df['10Y'] = pdr.DataReader('DGS10', 'fred', start, end)      #Fetches 10s Data from fred
df.dropna(inplace = True)
df['Spread'] = df['10Y'] - df['2Y']


In [18]:
#Creating a rolling window, a time period in which we want to compare results 
#Like for a 10 day Moving average, rolling window is 10,so it keeps on calculating 10 day MA for the last 10 days as we move on each day

window = 100
df['Spread_Mean'] = df['Spread'].rolling(window).mean()
df['Spread_Std'] = df['Spread'].rolling(window).std()
df['Z'] = (df['Spread'] - df['Spread_Mean'])/df['Spread_Std']

#Signal : Buy Steepner when z score is -1 and buy flattener when z score is 1, mean reversion trades in essense
df['Signal'] = np.where(df['Z'] < -1, 1, np.where(df['Z'] > 1, -1, 0))         #A multi question, where we are saying is z_score < -1, then signal is 1(steepener)

#Scale position size with z-score intensity( large size for large z-score because its a mean reversion strategy, and large outsized move means higher chnces of getting back to mean)
#It is capped at 3 so that we dont crazy size on any trade

df['Position'] = df['Signal']*np.clip(df['Z'].abs(),0,3)  #Taking absolute score of z and capping it to 3, so for z = 1.8, Position is 1.8 for signal -1 (Flattener)
                                                         #For z = -2.5, Position is 2,5 with Signal +1 (Steepener)


#Computing pnl using DV01 
DV01 = 100            # Assume for every 1bps move, 100 USD profit/loss          
df['Spread_Change'] = df['Spread'].diff()

#PnL = Positon on previous day * change in spread * DV01
df['PnL'] = df['Position'].shift(1) * df['Spread_Change'] * DV01
df['CumPnL'] = df['PnL'].cumsum()


#Is z_score is greater than 1, then signal is -1(flattener); otherwise 0(no trade)
# # 1) Pull the monthly series
ff = pdr.DataReader('FEDFUNDS', 'fred', start, end)

# Reindex onto your daily df index and forward-fill
df['FF'] = ff.reindex(df.index).ffill()

# (Optional) back-fill the very first days if you care  
# df['FF'].bfill(inplace=True))

# 2) Compute 63-day MA and the two regimes
df['FF_MA63']    = df['FF'].rolling(63).mean()
df['HikeRegime'] = df['FF'] > df['FF_MA63']    # good for steepeners
df['EaseRegime'] = df['FF'] < df['FF_MA63']    # good for flatteners

# 3) Gate your original z-score signal by regime
#    (df['Signal'] is +1 for steep, –1 for flat, 0 otherwise)
df['Signal_steep'] = np.where((df['Signal'] ==  1) & df['HikeRegime'],  1, 0)
df['Signal_flat']  = np.where((df['Signal'] == -1) & df['EaseRegime'],  -1, 0)
df['Signal_macro'] = df['Signal_steep'] + df['Signal_flat']

# 4) Position sizing (same z-score scaling/cap)
df['Position_macro'] = df['Signal_macro'] * np.clip(df['Z'].abs(), 0, 3)

# 5) PnL and cumulative PnL
df['PnL_macro']    = df['Position_macro'].shift(1) * df['Spread_Change'] * DV01
df['CumPnL_macro'] = df['PnL_macro'].cumsum()

# 6) Annualized Sharpe
sharpe_macro = df['PnL_macro'].mean() / df['PnL_macro'].std() * np.sqrt(252)
print("Sharpe with Macro Filter:", round(sharpe_macro, 2))

# 1) Do you ever get a non-zero macro signal?
print("Signal_macro counts:")
print(df['Signal_macro'].value_counts(dropna=False))

# 2) Peek at the first few rows where you *would* have entered
print("\nSample rows around potential entries:")
print(df[['Z','Signal','HikeRegime','EaseRegime','Signal_macro']].loc[
    (df['Signal'] != 0) | (df['Signal_macro'] != 0)
].head(10))

# 3) Check your filtered PnL
print("\nPnL_macro summary:")
print(df['PnL_macro'].describe())




Sharpe with Macro Filter: 0.0
Signal_macro counts:
Signal_macro
 0    949
 1    257
-1    138
Name: count, dtype: int64

Sample rows around potential entries:
                   Z  Signal  HikeRegime  EaseRegime  Signal_macro
DATE                                                              
2020-05-26  1.106487      -1       False       False             0
2020-05-28  1.203625      -1       False       False             0
2020-06-01  1.094784      -1       False       False             0
2020-06-02  1.006628      -1       False       False             0
2020-06-03  1.453484      -1       False       False             0
2020-06-04  1.745652      -1       False       False             0
2020-06-05  2.078148      -1       False       False             0
2020-06-08  1.828394      -1       False       False             0
2020-06-09  1.656141      -1       False       False             0
2020-06-10  1.243678      -1       False       False             0

PnL_macro summary:
count    1244.000